In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_excel("/content/drive/MyDrive/SML Project/Supply_Chain.xlsx")
data.head()

In [ ]:
data.info()

In [ ]:
data['order date']= pd.to_datetime(data['order date (DateOrders)'])
data['order year']=data['order date'].dt.year
data['order month']=data['order date'].dt.month
data['order day']=data['order date'].dt.day
data['order hour']=data['order date'].dt.hour
data['order minute']=data['order date'].dt.minute

In [ ]:
df = data.loc[:, ['order day','order hour','order minute','Order Status','Late_delivery_risk','Days for shipment (scheduled)', 'Department Name', 'Latitude', 'Longitude', 'Customer Segment', 'Market', 'Order Item Quantity', 'Shipping Mode', 'order_latitude', 'order_longitude']]
df.info()

In [ ]:
df['Order Fraud']= [1 if i=='SUSPECTED_FRAUD' else 0 for i in df['Order Status']]
df['Order Fraud'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer
for i in df.columns:
    if df[i].dtype=='object':
        df[i]=LabelEncoder().fit_transform(df[i])
df=df.dropna()
df.info()

In [ ]:
x = df.drop(["Late_delivery_risk", "Order Fraud", "Order Status"], axis=1).values
y = df[['Late_delivery_risk','Order Fraud']].values

In [ ]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, test_size = 0.2)

In [ ]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
m1=DecisionTreeClassifier()
m2=RandomForestClassifier()
m3=BaggingClassifier()
hybrid = StackingClassifier(estimators=[('dtc',m1),('rfc',m2),('bc',m3)],final_estimator=GradientBoostingClassifier())

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
model = MultiOutputClassifier(hybrid)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_train_pred = model.predict(x_train)

In [ ]:
accuracy_train1 = accuracy_score(y_train[:, 0], y_train_pred[:, 0])
percentage_accuracy_train1 = accuracy_train1 * 100
print("Training Accuracy for late delivery risk prediction: {:.2f}%".format(percentage_accuracy_train1))

In [ ]:
accuracy_train2 = accuracy_score(y_train[:, 1], y_train_pred[:, 1])
percentage_accuracy_train2 = accuracy_train2 * 100
print("Training Accuracy for fraud detection: {:.2f}%".format(percentage_accuracy_train2))

In [ ]:
y_test_pred = model.predict(x_test)

In [ ]:
accuracy1 = accuracy_score(y_test[:, 0], y_test_pred[:, 0])
percentage_accuracy1 = accuracy1 * 100
print(" Testing Accuracy for late delivery risk prediction: {:.2f}%".format(percentage_accuracy1))

In [ ]:
from sklearn.metrics import classification_report
report1 = classification_report(y_test[:, 0], y_test_pred[:, 0])
print("Classification report for late delivery risk prediction:\n", report1)

In [ ]:
y_pred_prob = model.predict_proba(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm1 = confusion_matrix(y_test[:, 0], y_test_pred[:, 0])
class_labels1 = ['Class 0', 'Class 1']

cmap = sns.diverging_palette(10, 150, s=90, l=60, center='light', as_cmap=True)

plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(cm1, annot=True, cmap=cmap, fmt='d', xticklabels=class_labels1, yticklabels=class_labels1)

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')

plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, thresholds = roc_curve(y_test[:, 0], y_pred_prob[0][:, 1])
roc_auc = roc_auc_score(y_test[:, 0], y_pred_prob[0][:, 1])

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
accuracy2 = accuracy_score(y_test[:, 1], y_test_pred[:, 1])
percentage_accuracy2 = accuracy2 * 100
print("Testing Accuracy for fraud detection: {:.2f}%".format(percentage_accuracy2))

In [ ]:
report2 = classification_report(y_test[:, 1], y_test_pred[:, 1])
print("Classification report for fraud detection:\n", report2)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm2 = confusion_matrix(y_test[:, 1], y_test_pred[:, 1])
class_labels2 = ['Class 0', 'Class 1']

cmap = sns.diverging_palette(10, 150, s=90, l=60, center='light', as_cmap=True)

plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(cm2, annot=True, cmap=cmap, fmt='d', xticklabels=class_labels2, yticklabels=class_labels2)

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')

plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

fpr, tpr, thresholds = roc_curve(y_test[:, 1], y_pred_prob[1][:, 1])
roc_auc = roc_auc_score(y_test[:, 1], y_pred_prob[1][:, 1])

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.005, 1.0])
plt.ylim([-0.005, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
x_test_v = sc.inverse_transform(x_test)
x_test_v = pd.DataFrame(x_test_v)
x_test_v.columns = ['order day','order hour','order minute','Days for shipment (scheduled)', 'Department Name', 'Latitude', 'Longitude', 'Customer Segment', 'Market', 'Order Item Quantity', 'Shipping Mode', 'order_latitude', 'order_longitude']
x_test_v = x_test_v.loc[:, ['order_latitude', 'order_longitude']]
x_test_v

In [ ]:
y_pred_v = pd.DataFrame(y_test_pred)
y_pred_v.columns = ['Late_delivery_risk','Order Fraud']
y_pred_v

In [ ]:
df_v = pd.concat([x_test_v, y_pred_v], axis=1)
df_v

In [ ]:
df_v.to_csv("/content/drive/MyDrive/SML Project/predicted_output.csv", index = False)

In [ ]:
!pip install folium

In [ ]:
import folium

m1 = folium.Map(location=[df_v["order_latitude"].mean(), df_v["order_longitude"].mean()], zoom_start=5)

# Define a function to set the marker color based on the result
def get_marker_color_late(result):
    return 'green' if result == 0 else 'red'
# Add markers to the map
for index, row in df_v.iterrows():
    folium.Marker(
        location=[row['order_latitude'], row['order_longitude']],
        popup=f"Result: {row['Late_delivery_risk']}",
        icon=folium.Icon(color=get_marker_color_late(row['Late_delivery_risk']))
    ).add_to(m1)

In [ ]:
m1.save("map1.html")

In [ ]:
m2 = folium.Map(location=[df_v["order_latitude"].mean(), df_v["order_longitude"].mean()], zoom_start=5)

# Define a function to set the marker color based on the result
def get_marker_color_late(result):
    return 'green' if result == 0 else 'red'
# Add markers to the map
for index, row in df_v.iterrows():
    folium.Marker(
        location=[row['order_latitude'], row['order_longitude']],
        popup=f"Result: {row['Order Fraud']}",
        icon=folium.Icon(color=get_marker_color_late(row['Order Fraud']))
    ).add_to(m2)

In [ ]:
m2.save("map2.html")